In [31]:
#######################################################################################
# Ryan Kladar Strategen Consulting Interview Case Study Part 2:
# Finding Volatility in the ERCOT price nodes particularly during congestion
#
# The goal of this analysis is to find the top dozen or so nodes that are the most
# volatile, as these will likely give us the best bang for our buck in arbitraging
# energy (buying/storing low and selling/dispatching high). Volatility is based on
# rolling daily range of price. The dozen nodes with the highest average daily range
# over the period are output and are good candidates for storage (if the DER Light or
# DER Heavy policy recommendations are instituted).
#######################################################################################
import csv
import pandas as pd
import numpy as np
import itertools

# First we name the columns in the csv for easier separation of columns of interest.
# This is clunky I know.
col_names = ['ISO_Name', 'Price_Node_Name', 'Price Node Type', 'ISO Price Node ID',
             'ISO Zone Name', 'Local_Datetime_HourEnding', 'Time Zone',
             'Price Type Full Description', 'Is Derived (Y/N)', 'Price',
             'Record Count', 'Substation Name', 'ISO Name ID', 'Price Node ID',
             'ISO Zone ID', 'Substation ID']

# Read in the csv file and give the column names to each column
data = pd.read_csv('ERCOT - (A-D) LMPs + Hubs (Day Ahead) - Trailing 370 days - '
                   '8.23.2017.csv', names=col_names, skiprows=[0])

# Make lists of the columns we care about, namely node name, datetime, and price at
# that datetime
price_nodes_list = data.Price_Node_Name.tolist()
date_time = data.Local_Datetime_HourEnding.tolist()
price = data.Price.tolist()

# Create an array of our three (now two) lists
col_that_matter = np.array((price_nodes_list, price))

# Sanity check on the shape of our array
print('We have this many rows, columns')
print(col_that_matter.shape)

# Peak inside the data by printing the number of total rows and the total number of
# unique nodes
price_nodes_set = set(price_nodes_list)
print('Number of total rows is '+str(len(price_nodes_list)))
print('Number of unique nodes is '+str(len(price_nodes_set)))

# Yeah, not specifying a dtype in each column makes the memory mad at me. Could be fixed by specifying the dtype of each
# column but if I specify it and there's an exception it'll break. I'd rather have the memory warning.

C:\Users\Kladar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We have this many rows, columns
(2, 28195917)
Number of total rows is 28195917
Number of unique nodes is 3238


In [47]:
print(col_that_matter) # Sanity Check aka does our data look right   
print(price[0:20]) # Another sanity check to see if prices are right. It's 1 am, alright?

[['_AK__AK_G1' '_AK__AK_G1' '_AK__AK_G1' ..., 'DYN_V_B' 'DYN_V_B' 'DYN_V_B']
 ['17.5' '16.7' '16.57' ..., '25.99' '23.89' '22.27']]


In [48]:
# Bear with me
from collections import defaultdict

# First we need to compare prices on a single node to see what the node is want to do, that is what the
# volatility in the node is. We can do this generally by looking at the range on a node over the entire period, standard 
# deviation on a node over the entire period, or by someother metric over the entire period. 

# We can also do this specifically by defining a period over which we care about volatility, in this case
# on a rolling 24 hour basis, meaning take each price one by one on a single node and compare it to the price 12 hours 
# prices before and after it. We can then define a "volatility event", say when the price changes more than $20 in 12 hours
# and add this to a counter for the node. The nodes with the most volatility events are the most volatile.

# Here we go. First lets do the general node analysis:

# Pretty sure creating node-based dictionaries of all the things I calculate will be the most straight forward method
node_dict=defaultdict(list) # Creates a dict that will use an empty list for new keys

for key, val in zip(price_nodes_list, price):
    node_dict[key].append(val)
    
#node_dict = dict(zip(price_nodes_list, price))
print(node_dict)


In [49]:
# Now that the dictionary has the node as the key with it prices as values,
# we can do basic statistics on them

import statistics
import scipy
import operator

# Just a small dictionary example to see if all the code in the block worked
# nodes = ['node1', 'node2', 'node3']
# prices = [[15.0, 10.0, 20.0], [5.0, 10.0, 0.0], [30.0, 10.0, 20.0] ]
# nodes_and_prices = zip(nodes, prices)
# node_dict = dict(zip(nodes, prices))

# Range - create a new dictionary with the range of prices of each node
range_dict = {}
for node in node_dict:
    range_dict[node] = [scipy.ptp(node_dict[node])]

# Since we want the top dozen nodes by range we need to sort it    
sorted_range = sorted(range_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    
# Standard Deviation - create a new dictionary with the standard deviation
# of each node
stdev_dict = {}
for node in node_dict:
    stdev_dict[node] = [statistics.stdev(node_dict[node])]

# Again, we have to sort them by standard deviation   
sorted_stdev = sorted(stdev_dict.items(), key=operator.itemgetter(1), reverse=True)
print('The nodes with the highest yearly range are: ')
print(sorted_range)



    

[('CORPSCHRSUBY', [4706.5200000000004]), ('ARMSTRON_L_A', [3987.27]), ('BUS1_138KV', [3977.3499999999999]), ('DIRSCOLLSUBZ', [3933.5300000000002]), ('DRSCOLS_AG1', [3933.5300000000002]), ('CENTRALSUB9X', [3265.0300000000002]), ('DIBOLL_8', [3236.8000000000002]), ('DIBOLSP_8X', [3236.8000000000002]), ('DIBOLSP_8Y', [3236.8000000000002]), ('DIBOLL_8U', [3236.8000000000002]), ('DIBOLL_8T', [3236.8000000000002]), ('DIBOLSP_8Z', [3236.8000000000002]), ('DIBOLSP_8', [3236.8000000000002]), ('CROCKETT_8W', [3200.1700000000001]), ('CROCKETT_8V', [3200.1700000000001]), ('CROCKETT_8', [3200.1700000000001]), ('BENBOLT_L_A', [2681.8799999999997]), ('5228_EB', [2529.9400000000001]), ('BARL_V_C', [2529.9400000000001]), ('5229_EB', [2529.9400000000001]), ('9BB3_EB', [2529.9400000000001]), ('BIGTRE_KB', [2462.7200000000003]), ('BIGTRE_KT', [2462.7200000000003]), ('BIGTRE_V_A', [2462.7200000000003]), ('BIGTRE_KU', [2462.7200000000003]), ('BIGTRE_L_A', [2462.7200000000003]), ('BLCBAYU4_L_A', [2462.720000

In [50]:
#Print the most variable nodes
print('The nodes with the highest standard deviation are: ')
print(sorted_stdev)

The nodes with the highest standard deviation are: 
[('ALPINE_MOB2', [119.99465033191424]), ('BARL_V_B', [110.02286959716662]), ('5233_EB', [110.02281573018338]), ('BARL_V_A', [106.88496876882132]), ('138_BUS2', [105.77960784327472]), ('ALMC_1_BUS1', [105.77201170893284]), ('5228_EB', [105.68631313173468]), ('BARL_V_C', [105.68631313173468]), ('5229_EB', [105.68631313173468]), ('9BB3_EB', [105.68631313173468]), ('ALMC_V_A', [105.52445637521016]), ('BRYR_L_A', [105.52445637521016]), ('BUS_02', [105.52445637521016]), ('CIEN_L_A', [105.52439280609978]), ('ACA_LD1', [105.52439280609978]), ('ALTUDA_L_A', [105.19203648110212]), ('ALPR_L_A', [105.19184668925817]), ('CCRK_L_A', [101.77623933678794]), ('BTP_LOAD1', [96.1172209383413]), ('BTP_LOAD2', [96.1172209383413]), ('AIR_LIQC_L_A', [92.02896876220245]), ('CORPSCHRSUBY', [79.50987179598164]), ('BLCBAYU4_L_A', [75.71400558965082]), ('BIGTRE_KB', [75.70190524783746]), ('BIGTRE_KT', [75.70190524783746]), ('BIGTRE_V_A', [75.70190524783746]), ('

In [ ]:
# Now here is where it gets tricky. Doing the rolling 24 hr volatility check
# on each node, counting the volatility events, and creating a sorted
# dictionary of them to see which nodes have the most volatility events
# is not trivial, especially when I have less time than a hackathon to do it
# 
# This is the ideal way to do it because then you can determine what is 
# the profitable delta_price and set that as your volatility event threshold.
# The node with the most volatility events is going to be your most profitable
# and you can probably put a lower bound on that probability, if current
# volitility event rates hold. You can also define the delta_price and
# delta_time separately so that we ca

# Pseudocode

1. loop through the values of price in each node ordered by datetime with a 
step of hour difference that you want (e.g. 6 hours), comparing each price.
If the price is greater than 30.0, add to a counter for the node. Add another
count to the counter for every 5.0 over that.

2. print out the nodes with the highest volatility event counters. Place storage 
there.

3. profit.

